In [1]:
!pip install torch==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install bitsandbytes==0.45.3
!pip install transformers==4.49.0
!pip install PyPDF2
!pip install rank_bm25


Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!pip install accelerate
!pip install safetensors
!pip install sentencepiece
!pip install google-colab


In [ ]:
import os
import PyPDF2

def extract_text_from_pdf(pdf_path):
    """
    استخراج متن از فایل PDF.
    """
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

from google.colab import files
uploaded = files.upload()
pdf_path = "/content/Drugs.pdf"  

document_text = extract_text_from_pdf(pdf_path)

def split_text_into_chunks(text, chunk_size=300):
    """
    تقسیم متن به چانک‌های کوچک برای بازیابی.
    """
    words = text.split()
    chunks = [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

chunks = split_text_into_chunks(document_text)
print(f"تعداد چانک‌ها: {len(chunks)}")

Saving Drugs.pdf to Drugs.pdf
تعداد چانک‌ها: 44


In [4]:
chunks

['دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهابزا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو بهصورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است: دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه. این دارو در اشکال دارویی قرص، کپسول، پماد، شیاف و آمپول برای استفاده خوراکی یا موضعی در تمام داروخانهها یافت می شود و بهمحض استفاده، دردهای حاد و ناگهانی نیازمند تسکین سریع درد را آرام میسازد. کاربرد شیاف دیکلوفناک از کاربردهای شیاف مسکن دیکلوفناک میتوان به این موارد اشاره کرد : • تسکین سردرد و میگرن: این شیاف جلوی حمالت میگرن را نمیگیرد، اما از درد طاقت فرسای آن میکاهد. • دنداندرد: بهترین مسکن موقتی برای تسکین درد ناشی عفونت یا کشیدن دندان، شیاف دیکلوفناک ۰۰۱ است. • زانودرد و درد مفاصل: کلیه عالئم آرتریت مفاصل شامل درد، التهاب، تورم و سفتی با شیاف دیکلوفناک بهطور موقت قابلتسکین است . • گلودرد و تب: شیاف دیکلوفناک اثر ضد درد و تببری دارد و دُزه

In [ ]:
import re
import numpy as np
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords_fa = set(["و", "در", "به", "از", "که", "این", "را", "با", "برای", "یک", "بر", "تا", "آن", "یا", "اما"])

digit_pattern = re.compile(r'\d+')
punctuation_pattern = re.compile(r'[^\w\s]')
extra_space_pattern = re.compile(r'\s+')

def preprocess_text(text):
    """
    پیش‌پردازش متن: حذف علائم نگارشی، تبدیل به حروف کوچک، حذف اعداد، حذف فواصل اضافی،
    جایگزینی حروف عربی، و حذف استاپ‌وردها.
    """
    text = text.lower()
    text = text.replace("ي", "ی").replace("ك", "ک") 
    text = digit_pattern.sub('', text) 
    text = punctuation_pattern.sub('', text)  
    text = extra_space_pattern.sub(' ', text).strip()

    words = text.split()
    filtered_text = " ".join([word for word in words if word not in stopwords_fa])

    return filtered_text

processed_chunks = [preprocess_text(chunk) for chunk in chunks]
tokenized_chunks = [chunk.split() for chunk in processed_chunks]

k1 = 1.6  
b = 0.7  
epsilon = 0.25  

bm25 = BM25Okapi(tokenized_chunks, k1=k1, b=b, epsilon=epsilon)

vectorizer = TfidfVectorizer(stop_words=list(stopwords_fa), smooth_idf=True, norm="l2", sublinear_tf=True)
tfidf_matrix = vectorizer.fit_transform(processed_chunks)


In [ ]:
import numpy as np

def retrieve_relevant_chunks(query, top_k=5, min_score=0.1):
    processed_query = preprocess_text(query)
    tokenized_query = processed_query.split()

    bm25_scores = bm25.get_scores(tokenized_query)

    query_vec = vectorizer.transform([processed_query])
    tfidf_scores = np.dot(tfidf_matrix, query_vec.T).toarray().flatten()

    combined_scores = (bm25_scores *0.5) + (tfidf_scores * 0.5)

    #print("امتیازهای BM25:", bm25_scores)
    #print("امتیازهای TF-IDF:", tfidf_scores)
    #print("امتیازهای ترکیبی:", combined_scores)

    top_indices = [i for i in np.argsort(combined_scores)[::-1] if combined_scores[i] > min_score][:top_k]

    #print("اندیس‌های انتخاب‌شده:", top_indices)
    #for i in top_indices:
       # print(f"چانک {i}: {chunks[i]} (امتیاز: {bm25_scores[i]})")

    if not top_indices:
        return [("متأسفم، اطلاعات مرتبط یافت نشد.", 0)]

    return [(chunks[i], combined_scores[i]) for i in top_indices]


In [ ]:

test_query = "اسامی تجاری آسپرین"
retrieved_chunks = retrieve_relevant_chunks(test_query)

if retrieved_chunks:
    best_chunk, score = retrieved_chunks[0]
    print(f"\nبهترین چانک بازیابی‌شده (امتیاز: {score}):")
    print(best_chunk)
else:
    print("هیچ چانک مرتبطی یافت نشد.")


بهترین چانک بازیابی‌شده (امتیاز: 4.051562682799997):
نگه دارید. ممکن است برندهای مختلف آسپرین شرایط نگهداری مختلفی داشته باشند. بستهبندی دارو را برای نحوه نگهداری همان برند بررسی کنید یا از داروساز سؤال کنید. آسپرین را در مکان گرم یا مرطوب نگهداری نکنید. از مصرف هرگونه محصول آسپرین که بویی شبیه سرکه تند میدهد خودداری کنید. همچنین، تمام داروها را از دسترس کودکان و حیوانات خانگی دور نگه دارید. از دفع دارو در سرویس بهداشتی یا سیستم فاضالب خودداری کنید. اگر تاریخ دارو گذشت یا دیگر به آن نیاز نداشتید در مورد دفع آن از داروساز یا سیستم فاضالب شهری سؤال بپرسید. اسامی تجاری آسپرین آسپرین با اسامی قرص آسپرین،آسپی کور، آسپرین ۱۸، آسپرین بایر، آسپرین پالس بایر و بوفرین تولید می شود. شرکتهای تولیدکننده • داروسازی پارس • داروسازی جالینوس • داروسازی اسوه • داروسازی تهران دارو • داروسازی خوارزمی • داروسازی حکیم • داروسازی تهران شیمی • داروسازی اکتوورکو


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "unsloth/llama-3-8b-bnb-4bit"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config
)

print(" مدل با موفقیت بارگذاری شد!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:206: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

✅ مدل با موفقیت بارگذاری شد!


In [ ]:
def generate_answer(query, context):
    """
    تولید پاسخ با استفاده از مدل LLaMA، جلوگیری از نمایش پاسخ نامرتبط.
    """
    query = query.strip()

    if not query:
        return "⚠ لطفاً یک سوال وارد کنید."

    if not context.strip():
        return "متأسفم، اما اطلاعاتی در مورد این سوال در متن موجود نیست."

    #print("متن بازیابی‌شده نهایی:")
    #print(context)

    prompt = (
        f"متن زیر را بخوان و فقط مرتبط‌ترین پاسخ را ارائه کن.\n\n"
        f"متن: {context}\n\n"
        f"سؤال: {query}\n\n"
        f"قوانین پاسخ‌دهی:\n"
        f"- مختصر و واضح باش.\n"
        f"- از اطلاعات نامرتبط اجتناب کن.\n"
        f"پاسخ:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.6, 
        top_p=0.7     
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    answer = generated_text.split("پاسخ:")[-1].strip()  
    answer = re.sub(r'\n+', ' ', answer)  
    answer = ' '.join(answer.split()[:100]) 
    words = answer.split()

    if len(words) > 100:
        answer = ' '.join(words[:100])

    if not any(word in answer for word in query.split()):
        return "متأسفم، به نظر می‌رسد پاسخ مدل با سوال شما مرتبط نیست."

    return answer

In [9]:
!pip install gradio
!pip install --upgrade gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
import gradio as gr
def qa_interface(inputs):
    """
    تابع واسط برای دریافت سوال فارسی، بازیابی چانک‌های مرتبط و تولید پاسخ.
    """

    relevant_chunks = retrieve_relevant_chunks(inputs)
    if relevant_chunks:
        context = "".join([chunk for chunk, score in relevant_chunks])
    else:
        context = "متأسفانه متن مرتبطی یافت نشد."

    answer = generate_answer(inputs, context)
    return answer

iface = gr.Interface(
    fn=qa_interface,
    inputs=gr.Textbox(lines=3, placeholder="سوال خود را به فارسی وارد کنید..."),
    outputs="text",
    title="سیستم پاسخگویی به سوالات (دارویی)",
    description="در این رابط کاربری، سوالات فارسی خود را وارد کنید و پاسخ تولید شده توسط مدل را دریافت نمایید."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://584391cbd41e73d096.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [19]:
test_data = [
    {"question": "کاربرد شیاف دیکلوفناک", "answer": "کاربردهای شیاف دیکلوفناک شامل تسکین دردهای میگرن، دندان‌درد، زانودرد، آرتریت، گلو درد، تب و درد قاعدگی است."},
    {"question": "نحوه مصرف کلیندامایسین تزریقی", "answer": "اگر فردی عفونت شدید داشته باشد و نتواند از سایر آنتی‌بیوتیک‌ها استفاده کند، ممکن است در بیمارستان کلیندامایسین تزریقی دریافت کند که به‌صورت درون عضلانی یا درون رگی استفاده می‌شود؛ ممکن است پزشک دُز اول را تزریق کند و نحوه تزریق دُزهای بعدی را آموزش دهد."},
    {"question": "عوارض جانبی عمومی ایندومتاسین", "answer": "ممکن است عوارضی مانند حالت تهوع، استفراغ، ناراحتی معده، گاز معده، سوزش سر دل، سردرد، خواب‌آلودگی یا سرگیجه رخ دهد؛ اگر هرکدام از این علائم ادامه یافت یا بدتر شد، سریعاً با پزشک یا داروساز تماس بگیرید و همچنین اگر بیماری شما تشدید شد، با پزشک مشورت کنید."},
    {"question": "نکات درباره خرید و استفاده از ایندومتاسین", "answer": "ایندوسین به‌صورت کپسول خوراکی (۲۵ و ۵۰ میلی‌گرم)، کپسول خوراکی با رهایش طولانی (۷۵ میلی‌گرم)، سوسپانسیون خوراکی و شیاف ایندومتاسین (۲۵، ۵۰ و ۱۰۰ میلی‌گرم) موجود است؛ دارو را در دمای اتاق، دور از نور و رطوبت نگهداری کنید و از قرار دادن آن در حمام خودداری نمایید، تمام داروها را از دسترس کودکان و حیوانات دور نگه دارید، از انجماد داروی مایع اجتناب کنید، دارو را در توالت یا فاضلاب نریزید مگر اینکه دستور داده شده باشد، و در صورت اتمام تاریخ مصرف یا عدم نیاز به دارو، آن را به‌درستی دفع کنید و در این مورد با داروساز یا شرکت محلی دفع زباله مشورت نمایید."},
    {"question": "آیا میتوان کلرفنیرامین را با داروی دیگری جایگزین کرد؟", "answer": "اگر پزشک برایتان کلرفنیرامین تجویز کرده است، بدون مشورت با او دارویتان را تغییر ندهید؛ از جمله داروهای مشابه کلرفنیرامین می‌توان به پردنیزون، هیدروکسی‌زین، فلوتیکازون، مونته‌لوکاست، سیتریزین، لوراتادین و تریامسینولون اشاره کرد."},
    {"question": "عوارض جانبی تریمیپرامین", "answer": "همه داروها عوارض دارند و پزشکان با توجه به مزایای آنها نسبت به عوارض، دارو را تجویز می‌کنند؛ عوارض تریمیپرامین شامل سه دسته است: ۱. **عوارض رایج** مانند سرگیجه، خواب‌آلودگی، مشکل در دفع ادرار، سردرد، ضعف، تغییر اشتها یا وزن، خشکی دهان، تاری دید و یبوست که در صورت آزاردهنده بودن یا تشدید، باید با پزشک مشورت شود. ۲. **عوارض کمتر شایع** مانند تغییرات خلقی (گیجی، افسردگی، اضطراب)، بی‌حسی یا گزگز دست و پا، زنگ زدن گوش، مشکلات جنسی، رعشه، استفراغ یا یبوست شدید، سردرد شدید، درد، قرمزی یا ورم دست و پا که باید فوراً به پزشک اطلاع داده شود. ۳. **عوارض خطرناک** مانند بینظمی ضربان قلب، سرگیجه شدید، از حال رفتن، تشنج، مشکل تکلم، ضعف یک طرفه بدن، درد یا ورم چشم، گشادی مردمک‌ها، مشکل بینایی (مانند دیدن رنگین‌کمان اطراف نورها)، درد فک، قفسه سینه یا دست چپ که در صورت بروز، باید فوراً به اورژانس مراجعه کرد."},
    {"question": "آسپرین چیست و چه کاربردی دارد؟", "answer": "آسپرین (Aspirin) یک علامت تجاری متعلق به شرکت داروسازی آلمانی بایر است که با نام عمومی استیل سالیسیلیک اسید یا آ اس آ (ASA) شناخته می‌شود؛ این دارو یک داروی ضدالتهابی غیراستروئیدی رایج و اولین رده از این داروها است که برای تسکین درد، تب خفیف، سردرد، کاهش ورم و به‌عنوان داروی ضدالتهابی و رقیق‌کننده خون استفاده می‌شود و می‌تواند به‌صورت تجویزی یا بدون نسخه تهیه شود. قرص آسپرین حاوی سالیسیلات است، ترکیبی که در درخت بید و مورد وجود دارد و اولین بار حدود ۴۰۰۰ سال پیش کشف شد؛ بقراط از پوست درخت بید برای تسکین درد و تب استفاده می‌کرد و برخی افراد هنوز از پوست درخت بید برای درمان طبیعی سردرد و درد خفیف استفاده می‌کنند."},
    {"question": "موارد منع مصرف آسپرین", "answer": "اگر به آسپرین، سالیسیلات‌ها، داروهای ضدالتهابی غیراستروئیدی (مثل ایبوپروفن) یا مواد دیگر حساسیت دارید، قبل از مصرف با پزشک مشورت کنید؛ در صورت داشتن مشکلاتی مانند اختلالات خونریزی، بیماری‌های کلیه، کبد، معده، آسم، پولیپ بینی، بیماری قلبی، فشار خون بالا، نقرس، کمبود آنزیم‌های خاص یا کم‌خونی، مصرف آسپرین را با پزشک در میان بگذارید. آسپرین ممکن است باعث خونریزی معده شود و مصرف الکل و سیگار خطر عوارض را افزایش می‌دهد. کودکان و نوجوانان زیر ۱۸ سال در صورت ابتلا به آبله مرغان، آنفولانزا یا پس از واکسیناسیون نباید آسپرین مصرف کنند، زیرا خطر سندرم ری (یک بیماری نادر اما جدی) افزایش می‌یابد. سالمندان نیز به عوارضی مانند خونریزی یا زخم معده حساسترند. قبل از جراحی، تمام داروهای مصرفی را به پزشک اطلاع دهید."},
    {"question": "", "answer": "⚠ لطفاً یک سوال وارد کنید."},
    {"question": "حنانه", "answer": "متأسفم، به نظر می‌رسد پاسخ مدل با سوال شما مرتبط نیست."},
    # ...
]


In [20]:
### توابع ارزیابی ضروری ###
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def exact_match(pred_answer, true_answer):
    """محاسبه Exact Match (EM)"""
    pred_clean = preprocess_text(pred_answer)
    true_clean = preprocess_text(true_answer)
    return int(pred_clean == true_clean)

def calculate_f1(pred_answer, true_answer):
    """محاسبه F1-Score"""
    pred_tokens = set(preprocess_text(pred_answer).split())
    true_tokens = set(preprocess_text(true_answer).split())

    if len(pred_tokens) == 0 or len(true_tokens) == 0:
        return 0.0

    tp = len(pred_tokens & true_tokens)
    precision = tp / len(pred_tokens)
    recall = tp / len(true_tokens)

    if (precision + recall) == 0:
        return 0.0

    return 2 * (precision * recall) / (precision + recall)

### تابع ارزیابی تفصیلی ###
def detailed_evaluation(test_data):
    total_em = 0
    total_f1 = 0
    results = []

    for idx, item in enumerate(test_data, 1):
        question = item["question"]
        true_answer = item["answer"]

        # تولید پاسخ
        relevant_chunks = retrieve_relevant_chunks(question)
        context = " ".join([chunk for chunk, _ in relevant_chunks])
        pred_answer = generate_answer(question, context)

        # محاسبه معیارها
        em = exact_match(pred_answer, true_answer)
        f1 = calculate_f1(pred_answer, true_answer)

        # ذخیره نتایج
        results.append({
            "id": idx,
            "question": question,
            "pred_answer": pred_answer,
            "true_answer": true_answer,
            "EM": em,
            "F1": f1
        })

        total_em += em
        total_f1 += f1

    # میانگین
    avg_em = total_em / len(test_data)
    avg_f1 = total_f1 / len(test_data)

    return {
        "overall": {"EM": avg_em, "F1": avg_f1},
        "details": results
    }

### اجرای ارزیابی ###
evaluation_results = detailed_evaluation(test_data)

### نمایش نتایج ###
print("\n" + "="*50)
print(f"میانگین Exact Match (EM): {evaluation_results['overall']['EM']:.2f}")
print(f"میانگین F1-Score: {evaluation_results['overall']['F1']:.2f}")
print("="*50 + "\n")

# نمایش جزئیات
for item in evaluation_results["details"]:
    print("\n" + "-"*50)
    print(f"ID: {item['id']}")
    print(f"سوال: {item['question']}")
    print(f"پاسخ واقعی:\n{item['true_answer']}")
    print(f"پاسخ مدل:\n{item['pred_answer']}")
    print(f"EM: {item['EM']} | F1: {item['F1']:.2f}")


میانگین Exact Match (EM): 0.20
میانگین F1-Score: 0.50


--------------------------------------------------
ID: 1
سوال: کاربرد شیاف دیکلوفناک
پاسخ واقعی:
کاربردهای شیاف دیکلوفناک شامل تسکین دردهای میگرن، دندان‌درد، زانودرد، آرتریت، گلو درد، تب و درد قاعدگی است.
پاسخ مدل:
آمپول دیکلوفناک بهعنوان مسکن ضدالتهابی با ترکیبات مشابه قرص، کپسول و شیاف همین دارو، می تواند
EM: 0 | F1: 0.19

--------------------------------------------------
ID: 2
سوال: نحوه مصرف کلیندامایسین تزریقی
پاسخ واقعی:
اگر فردی عفونت شدید داشته باشد و نتواند از سایر آنتی‌بیوتیک‌ها استفاده کند، ممکن است در بیمارستان کلیندامایسین تزریقی دریافت کند که به‌صورت درون عضلانی یا درون رگی استفاده می‌شود؛ ممکن است پزشک دُز اول را تزریق کند و نحوه تزریق دُزهای بعدی را آموزش دهد.
پاسخ مدل:
کلیندامایسین را دقیقا براساس دستور پزشک مصرف کنید. از دستورالعمل همراه دارو پیروی کنید و راهنمای دارو را مطالعه نمایید. نحوه مصرف کپسول و گرانول های خوراکی کلیندامایسین روش مصرف کپسول برای بیماران عادی کپسول کلیندامایسین را بهصورت دهانی قورت دهید.